In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
#select works to select from an item
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
import spacy
from selenium.common.exceptions import NoSuchElementException

In [2]:
#To simulate the delay
import random
import time
start_time = time.time()
normal_delay = random.normalvariate(2, 0.5)
time.sleep(normal_delay)    
# print("--- %.5f seconds ---" % (time.time() - start_time))

def delay(t):
    normal_delay = random.normalvariate(t, 0.5)
    time.sleep(normal_delay)

In [10]:
# Run the Chrome Driver
driver = webdriver.Chrome(executable_path=r'/chromedriver')

In [11]:
# Let's go the main page of Amazon website
driver.get('http://www.amazon.com')
#We can get access the full page source with this line of code
"""
driver.page_source
"""
# Let's find the search field and click on it to make it ready to type on
delay(4)
search_field = driver.find_element_by_id('twotabsearchtextbox')
search_field.click()
delay(3)

In [12]:
# Here we feed the keyword in search field
search_field.send_keys('AKG headphones')
search_field.send_keys(Keys.ENTER)
delay(3)

In [13]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index
            index += 1
    return -1

In [14]:
# find headphone feature on amazon page and narrow down searches into wireless headphones only
wireless = driver.find_element_by_xpath('//*[@id="leftNavContainer"]/ul[7]/div/li[4]/span/span/div/label/span/span')
wireless.click()
delay(3)

In [15]:
# current page to html
current_page = driver.page_source
current_page_html = BS(current_page, 'html5lib')
# find ul list containing products by id
ul_list = current_page_html.find("ul", {"id": "s-results-list-atf"})
# initialize asin list to store all asin and product link to store all product links
asin_list = []
product_list = []
for li in ul_list:
    asin = li['data-asin']
    asin_list.append(asin)
    a_tag = li.find('a', href=True)['href']
    product_list.append(a_tag)

In [38]:
for asin, prod in zip(asin_list, product_list):
    driver.get(prod)
    delay(3)
    # spot [see all customer reviews]
    try:
        all_cust_reviews_link = driver.find_element_by_id('reviewSummary').find_element_by_id('dp-summary-see-all-reviews')
        all_cust_reviews_link.click()
        delay(3)
        # get review page url
        current_url = driver.current_url
        index= find_str(current_url, '&reviewerType')
        string = current_url[0:index] + "&pageNumber=" + current_url[index:]
        # current page to HTML
        all_cust_reviews_page = driver.page_source
        all_cust_reviews_soup = BS(all_cust_reviews_page, 'html5lib')
        # get total review counts
        review_counts = all_cust_reviews_soup.find('span', attrs={'data-hook': 'total-review-count'}).get_text()
        # remove , in counts
        review_counts = int(review_counts.replace(',',''))
        # # of pages
        num_of_page = int(np.ceil(review_counts / 10))

        add_eacpage_score_list = []
        add_eacpage_date_list = []
        add_eacpage_title_list = []
        add_eacpage_review_list = []
        add_eacpage_vote_list = []

        for i in range(1,num_of_page+2):
            # define page link
            review_page_link = current_url[0:index] + "&pageNumber=" + str(i) + current_url[index:]
            driver.get(review_page_link)
            delay(2)
            # current page to HTML
            all_cust_reviews_page = driver.page_source
            all_cust_reviews_soup = BS(all_cust_reviews_page, 'html5lib')
            # get div containing all reviews body
            reviews_soup_div = all_cust_reviews_soup.find('div', attrs={'id':'cm_cr-review_list'})
            reviews_soup_div_data = reviews_soup_div.find_all('div', attrs={'class': 'a-section review'})
            num_of_reviews = len(reviews_soup_div_data)
            # for loop through reviews_soup_div_data
            score_list = []
            date_list = []
            title_list = []
            review_list = []
            vote_list = []

            for i in range(0, num_of_reviews):
                review = reviews_soup_div_data[i].find('div', attrs={'class': 'a-section celwidget'})
                # retrieve title score and append it to score_list
                score = review.find('a', attrs={'class': 'a-link-normal'})['title'].encode("utf-8")
                score_list.append(score)
                # retrieve date
                date = review.find('span', attrs={'data-hook': 'review-date'}).get_text().encode("utf-8")
                date_list.append(date)
                # retrieve title and append it to title_list
                title = review.find('a', attrs={'data-hook': 'review-title'}).get_text().encode("utf-8")
                title_list.append(title)
                # get review data and store in review_list
                review_body = review.find('span', attrs={'data-hook': 'review-body'}).get_text().encode("utf-8")
                review_list.append(review_body)
                # get [how many people found helpful]
                vote = ""
                if review.find('span', attrs={'data-hook': 'helpful-vote-statement'}):
                    vote = review.find('span', attrs={'data-hook': 'helpful-vote-statement'}).get_text().encode("utf-8")
                vote_list.append(vote)
            add_eacpage_score_list.extend(score_list)
            add_eacpage_date_list.extend(date_list)
            add_eacpage_title_list.extend(title_list)
            add_eacpage_review_list.extend(review_list)
            add_eacpage_vote_list.extend(vote_list)

        df = pd.DataFrame({'Title':add_eacpage_title_list, 'Date': add_eacpage_date_list, 'Score': add_eacpage_score_list, 'Review Contents': add_eacpage_review_list, 'How Helpful': add_eacpage_vote_list})
    #     df.to_csv(asin + '.csv')
        df.to_csv(asin + ".csv")
    except NoSuchElementException:
        driver.execute_script("window.history.go(-1)")